In [1]:
import pandas as pd
import talib as ta
import matplotlib.pyplot as plt
import numpy as np

import plotly 
import plotly.io as pio
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

#### RSI + MFI + Stochastic

In [94]:
#Forex
colnames = ['Data','Open','High','Low','Close','Vol']
df = pd.read_csv('./Forex Data/EURUSD_eatradingacademy/EURUSD1.csv', sep = '\t', header=None, names = colnames)
df = pd.read_csv('./Forex Data/EURUSD_eatradingacademy/EURUSD1.csv', sep = '\t', header=None, names = colnames)

In [95]:
#Ação yfinance

In [96]:
#df = df.iloc[:10000]

In [97]:
'''df['MA_50'] = ta.SMA(df['Close'],timeperiod=50)
df['EMA'] = ta.EMA(df['Close'], timeperiod = 5)
df['BB_up'],df['BB_mid'],df['BB_low'] = ta.BBANDS(df['Close'], timeperiod=20, nbdevup=2, nbdevdn=2, matype=0)
df['TEMA_S'] = ta.TEMA(df['Close'], timeperiod = 12)
df['TEMA_L'] = ta.TEMA(df['Close'], timeperiod = 36)
df['Tamanho_candle'] = abs(df['Close'] - df['Open'])
df['Tamanho_total'] = abs(df['High'] - df['Low'])'''

def tendencia(x):
    if x['Close'] > x['MA_200']:
        return 'Alta'
    else:
        return 'Baixa'

def mfi_rsi_stochastic_tendencia(x):
    if x['RSI'] < 30 and x['MFI'] < 20 and x['slowk'] < 20 and x['Tendencia'] == 'Alta':
        return 'call'
    elif x['RSI'] > 70 and x['MFI'] > 80 and x['slowk'] > 80 and x['Tendencia'] == 'Baixa':
        return 'sell' 
    else:
        return 0

def resultado_var(x):
    if x['pos_mfi_rsi_stoch_tend'] == 'call':
        resultado = x['close_distancia']/x['Close']
        return resultado
    elif x['pos_mfi_rsi_stoch_tend'] == 'sell':
        resultado = x['Close']/ x['close_distancia']
        return resultado
    else:
        return 0

def resultado_qtd(x):
    if x['pos_mfi_rsi_stoch_tend'] == 'call':
        resultado = x['close_distancia'] - x['Close']
        return resultado
    elif x['pos_mfi_rsi_stoch_tend'] == 'sell':
        resultado = x['Close'] - x['close_distancia']
        return resultado
    else:
        return 0

tempo_distancia = -10

df['MA_200'] = ta.SMA(df['Close'],timeperiod=200)
df['RSI'] = ta.RSI(df['Close'],14)
df['MFI'] = ta.MFI(df['High'], df['Low'], df['Close'], df['Vol'], timeperiod=14)
df['slowk'], df['slowd'] = ta.STOCH(df['High'], df['Low'], df['Close'], fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
df['Tendencia'] = df.apply(tendencia,axis=1)
df['pos_mfi_rsi_stoch_tend'] = df.apply(mfi_rsi_stochastic_tendencia, axis = 1)
df['close_distancia'] = df['Close'].shift(tempo_distancia)
df['Resultado_var'] = df.apply(resultado_var, axis = 1)
df['Resultado_qtd'] = df.apply(resultado_qtd, axis = 1)


In [98]:
qtd_compras = df[(df['Resultado_var'] != 0) & (df['pos_mfi_rsi_stoch_tend'] == 'call')].shape[0]
qtd_compras_lucro = df[(df['Resultado_var'] != 0) & (df['pos_mfi_rsi_stoch_tend'] == 'call') & (df['Resultado_qtd'] > 0)].shape[0]

print(f'Total de compras: {qtd_compras} \nTotal de compras com lucro: {qtd_compras_lucro} \nPorcentagem de acerto:  {100 * qtd_compras_lucro/qtd_compras}%')

Total de compras: 161 
Total de compras com lucro: 84 
Porcentagem de acerto:  52.17391304347826%


In [99]:
qtd_compras = df[(df['Resultado_var'] != 0) & (df['pos_mfi_rsi_stoch_tend'] == 'sell')].shape[0]
qtd_compras_lucro = df[(df['Resultado_var'] != 0) & (df['pos_mfi_rsi_stoch_tend'] == 'sell') & (df['Resultado_qtd'] > 0)].shape[0]

print(f'Total de vendas: {qtd_compras} \nTotal de vendas com lucro: {qtd_compras_lucro} \nPorcentagem de acerto:  {100 * qtd_compras_lucro/qtd_compras}%')

Total de vendas: 146 
Total de vendas com lucro: 91 
Porcentagem de acerto:  62.32876712328767%
